In [ ]:
# Install required packages (if not already installed)
import sys
print('Python executable:', sys.executable)
!{sys.executable} -m pip install --quiet palmerpenguins seaborn plotly nbconvert || true

Python executable: /workspaces/icb_slide/.venv/bin/python


487.06s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [ ]:
# Imports and plotting setup
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob

sns.set(style='whitegrid', context='notebook', palette='deep')
plt.rcParams['figure.dpi'] = 150

os.makedirs('figures', exist_ok=True)
os.makedirs('tables', exist_ok=True)
print('Ready. Figures will be saved in ./figures')

In [ ]:
# Load penguins dataset (seaborn fallback to palmerpenguins)
try:
    df = sns.load_dataset('penguins')
    print('Loaded with seaborn')
except Exception as e:
    print('seaborn load failed, trying palmerpenguins')
    from palmerpenguins import load_penguins
    df = load_penguins()

# Save a local copy
df.to_csv('penguins.csv', index=False)
print('Data shape:', df.shape)
df.head()

Loaded with seaborn
Data shape: (344, 7)


,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female


In [5]:
# Basic EDA: head/info/describe/value_counts/missing
print('Columns:', df.columns.tolist())
print('\nInfo:')
df.info()

print('\nDescribe:')
df.describe(include='all')

print('\nSpecies counts:')
print(df['species'].value_counts(dropna=False))

print('\nMissing values:')
print(df.isnull().sum())

Columns: ['species', 'island', 'bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g', 'sex']

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            344 non-null    object 
 1   island             344 non-null    object 
 2   bill_length_mm     342 non-null    float64
 3   bill_depth_mm      342 non-null    float64
 4   flipper_length_mm  342 non-null    float64
 5   body_mass_g        342 non-null    float64
 6   sex                333 non-null    object 
dtypes: float64(4), object(3)
memory usage: 18.9+ KB

Describe:

Species counts:
species
Adelie       152
Gentoo       124
Chinstrap     68
Name: count, dtype: int64

Missing values:
species               0
island                0
bill_length_mm        2
bill_depth_mm         2
flipper_length_mm     2
body_mass_g           2
sex                  

In [6]:
# Missing value handling and preprocessing
# We'll drop rows with missing values for simplicity (alternative: impute)
df_clean = df.dropna().reset_index(drop=True)
print('Shape after dropping NA:', df_clean.shape)
print(df_clean.isnull().sum())

# Normalize categorical types
for col in ['species','island','sex']:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].astype('category')

# Derived variable
df_clean['bill_ratio'] = df_clean['bill_length_mm'] / df_clean['bill_depth_mm']
print('\nDerived column `bill_ratio` added. Sample:')
df_clean[['bill_length_mm','bill_depth_mm','bill_ratio']].head()

Shape after dropping NA: (333, 7)
species              0
island               0
bill_length_mm       0
bill_depth_mm        0
flipper_length_mm    0
body_mass_g          0
sex                  0
dtype: int64

Derived column `bill_ratio` added. Sample:


,bill_length_mm,bill_depth_mm,bill_ratio
0,39.1,18.7,2.090909
1,39.5,17.4,2.270115
2,40.3,18.0,2.238889
3,36.7,19.3,1.901554
4,39.3,20.6,1.907767


In [7]:
# 1) Univariate plots: histograms, boxplots, violin plots
num_cols = ['bill_length_mm','bill_depth_mm','flipper_length_mm','body_mass_g','bill_ratio']

for i, col in enumerate(num_cols, start=1):
    plt.figure(figsize=(6,4))
    sns.histplot(df_clean[col], kde=True, bins=20)
    plt.title(f'Histogram: {col}')
    fname = f'figures/{i:02d}_hist_{col}.png'
    plt.savefig(fname)
    plt.close()

# Boxplots and violin plots per species
plt.figure(figsize=(6,4))
sns.boxplot(data=df_clean, x='species', y='body_mass_g')
plt.title('Boxplot: body_mass_g by species')
plt.savefig('figures/box_body_mass_by_species.png')
plt.close()

plt.figure(figsize=(6,4))
sns.violinplot(data=df_clean, x='species', y='bill_length_mm')
plt.title('Violin: bill_length_mm by species')
plt.savefig('figures/violin_bill_length_by_species.png')
plt.close()

print('Saved univariate plots.')

Saved univariate plots.


In [8]:
# 2) Bivariate plots: scatterplots, regplot, jointplot
plt.figure(figsize=(6,4))
sns.scatterplot(data=df_clean, x='bill_length_mm', y='bill_depth_mm', hue='species')
plt.title('Bill length vs depth by species')
plt.savefig('figures/scatter_bill_length_depth_by_species.png')
plt.close()

plt.figure(figsize=(6,4))
sns.scatterplot(data=df_clean, x='bill_length_mm', y='bill_depth_mm', hue='species', size='flipper_length_mm', sizes=(20,200), alpha=0.8)
plt.title('Bill vs depth (size=flipper_length_mm)')
plt.savefig('figures/scatter_bill_vs_depth_size.png')
plt.close()

# regplot
plt.figure(figsize=(6,4))
sns.regplot(data=df_clean, x='bill_length_mm', y='bill_depth_mm', scatter_kws={'s':15}, line_kws={'color':'red'})
plt.title('Regression: bill_length vs bill_depth')
plt.savefig('figures/regplot_bill_length_vs_depth.png')
plt.close()

# jointplot
jp = sns.jointplot(data=df_clean, x='bill_length_mm', y='body_mass_g', kind='reg', height=6)
jp.fig.suptitle('Jointplot: bill_length_mm vs body_mass_g', y=1.02)
jp.fig.savefig('figures/jointplot_bill_length_body_mass.png')
plt.close()

print('Saved bivariate plots.')

Saved bivariate plots.


In [9]:
# 3) Multivariate: pairplot and heatmap of correlations
pair_vars = ['bill_length_mm','bill_depth_mm','flipper_length_mm','body_mass_g']
pp = sns.pairplot(df_clean, hue='species', vars=pair_vars)
pp.fig.suptitle('Pairplot of numeric features', y=1.02)
pp.fig.savefig('figures/pairplot_numeric.png')
plt.close()

corr = df_clean[pair_vars + ['bill_ratio']].corr()
plt.figure(figsize=(6,4))
sns.heatmap(corr, annot=True, cmap='vlag', fmt='.2f')
plt.title('Correlation heatmap')
plt.savefig('figures/heatmap_correlation.png')
plt.close()

print('Saved pairplot and heatmap.')

Saved pairplot and heatmap.


In [13]:
# 4) Categorical bar charts and tables (crosstab & pivot)
# Countplot for species
plt.figure(figsize=(6,4))
sns.countplot(data=df_clean, x='species')
plt.title('Count of penguins by species')
plt.savefig('figures/count_species.png')
plt.close()

# Crosstab: species x island
crosstab_species_island = pd.crosstab(df_clean['species'], df_clean['island'])
with open('tables/crosstab_species_island.md','w') as f:
    f.write(crosstab_species_island.to_markdown())

# Pivot table: mean body_mass by island x species
pivot_island_species_bodymass = pd.pivot_table(df_clean, index='island', columns='species', values='body_mass_g', aggfunc='mean')
with open('tables/pivot_island_species_bodymass.md','w') as f:
    f.write(pivot_island_species_bodymass.round(1).to_markdown())

# Grouped bar (sex by species)
ct = pd.crosstab(df_clean['species'], df_clean['sex'])
ax = ct.plot(kind='bar', figsize=(6,4))
plt.title('Sex count by species')
plt.ylabel('count')
plt.tight_layout()
plt.savefig('figures/grouped_bar_sex_species.png')
plt.close()

# Stacked/normalized island by species
ct2 = pd.crosstab(df_clean['island'], df_clean['species'])
ct2_norm = ct2.div(ct2.sum(axis=1), axis=0)
ct2_norm.plot(kind='bar', stacked=True, figsize=(6,4))
plt.title('Normalized island composition by species')
plt.ylabel('proportion')
plt.tight_layout()
plt.savefig('figures/stacked_bar_island_species.png')
plt.close()

# Save textual copies of tables to additional files
crosstab_species_island.to_csv('tables/crosstab_species_island.csv')
pivot_island_species_bodymass.to_csv('tables/pivot_island_species_bodymass.csv')

print('Saved categorical charts and tables.')

/tmp/ipykernel_9270/3369361809.py:15: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot_island_species_bodymass = pd.pivot_table(df_clean, index='island', columns='species', values='body_mass_g', aggfunc='mean')


Saved categorical charts and tables.


In [11]:
# Install missing optional dependencies
import sys
!{sys.executable} -m pip install --quiet tabulate || true

185.31s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [ ]:
# 5) Ensure at least 10 images saved and produce a markdown report
images = sorted(glob.glob('figures/*.png'))
print('Found images:', len(images))

# Read the tables
ct_md = crosstab_species_island.to_markdown()
pivot_md = pivot_island_species_bodymass.round(1).to_markdown()

# Build markdown content
md_lines = [
    '# Penguins 데이터셋 분석 보고서\n',
    '이 보고서는 펭귄 데이터셋을 EDA하고 시각화를 통해 주요 패턴을 요약합니다.\n',
    '## 생성된 그래프\n'
]
for img in images:
    bname = os.path.basename(img)
    md_lines.append(f'![]({img})\n')
    # Add insight text (Korean) under each image if available
    ins = insights.get(bname, default_insight)
    # Ensure paragraph formatting
    md_lines.append('\n**인사이트:**\n')
    md_lines.append(ins + '\n')

md_lines.append('\n## 교차표: species x island\n')
md_lines.append(ct_md + '\n')
md_lines.append('\n**교차표 인사이트:**\n')
md_lines.append(insights.get('crosstab_species_island.md', default_insight) + '\n')

md_lines.append('\n## 피봇테이블: mean body_mass_g (island x species)\n')
md_lines.append(pivot_md + '\n')
md_lines.append('\n**피봇테이블 인사이트:**\n')
md_lines.append(insights.get('pivot_island_species_bodymass.md', default_insight) + '\n')

md_lines.append('\n## 간단 결론\n')
md_lines.append('- 데이터는 세 종(species) 간 체질 차이가 존재합니다.\n')
md_lines.append('- 섬(island)과 종(species) 분포는 상이합니다.\n')

# Write markdown file
with open('penguins_analysis.md','w', encoding='utf-8') as f:
    f.write('\n'.join(md_lines))

print('Wrote penguins_analysis.md with', len(images), 'images')

Found images: 16
Wrote penguins_analysis.md with 16 images


In [ ]:
# 5a) Detailed Korean insights for each figure and table (500~1000자 권장)
# Map image filename (basename) to Korean insight text
insights = {
    '01_hist_bill_length_mm.png': (
        "펭귄의 부리 길이(bill_length_mm) 분포는 좌우 어느 한 쪽으로 크게 치우치지 않고 중앙에 모여 있으나, 종(species)별로 평균과 분포의 차이를 보입니다. \n"
        "Adelie와 Chinstrap는 비교적 짧은 부리 길이 쪽에 몰려 있는 반면, Gentoo는 분포가 더 오른쪽(긴 부리)으로 이동해 평균이 큽니다.\n"
        "이 분포는 종 간의 생태적 적응이나 먹이 섭취 방식의 차이를 시사합니다: 긴 부리는 다른 먹이 취득 전략이나 먹이 종류와 연결될 가능성이 있습니다.\n"
        "또한 히스토그램의 꼬리와 이상치(특이값)을 확인하면 샘플링 또는 측정 과정에서의 변동을 평가하는 데 도움이 됩니다.\n"
        "결론적으로 부리 길이는 종 구분에 유용한 형태학적 특징이며, 추가적으로 섬(island) 또는 성별(sex)에 따른 분할 분석을 하면 더 명확한 패턴이 나타날 수 있습니다."
    ),
    '02_hist_bill_depth_mm.png': (
        "부리 깊이(bill_depth_mm)의 분포는 일부 종에서 상대적으로 큰 값을 가지며, 종 간 차이가 관측됩니다.\n"
        "특히 Adelie와 Chinstrap는 중간-작은 깊이 범위에 밀집하고, Gentoo는 평균적으로 깊이가 다른 그룹에 비해 더 얕거나 뚜렷히 분리되는 패턴을 보일 수 있습니다.\n"
        "부리 깊이는 먹이 선택 및 포획 방식, 그리고 서식지의 먹이자원 특성과 관련될 수 있으므로 생태적 해석의 실마리를 제공합니다.\n"
        "히스토그램의 모양(예: 다봉성, 편향성)과 극단값을 확인하면 개체군 내 이형성(variation)과 모집단 구조를 이해하는 데 도움이 됩니다."
    ),
    '03_hist_flipper_length_mm.png': (
        "핀 길이(flipper_length_mm)는 이동 능력과 관련된 신체 특성으로, 히스토그램은 종별로 뚜렷한 차이를 보이는 편입니다.\n"
        "Gentoo는 다른 종에 비해 긴 플리퍼를 가지는 경향이 있어 수영 성능이나 먹이 포착 전략 차이를 시사합니다.\n"
        "분포의 폭과 꼬리를 보면 연령대나 성차에 따른 이질성(heterogeneity)도 살펴볼 필요가 있습니다.\n"
        "추가로 성별로 나누어 플리퍼 길이를 비교하면 성적 이형성(sexual dimorphism) 여부를 정량적으로 확인할 수 있습니다."
    ),
    '04_hist_body_mass_g.png': (
        "체질량(body_mass_g) 분포는 세 종 간에 가장 뚜렷한 차이를 보이는 변수 중 하나입니다.\n"
        "Gentoo는 평균 체질량이 높아 오른쪽으로 치우쳐 있고, Adelie와 Chinstrap는 상대적으로 낮은 체질량을 보입니다.\n"
        "체질량 차이는 먹이 섭취, 대사, 생리적 적응 및 번식 전략과 관련될 수 있으며, 서식지 자원 분포와도 연동될 가능성이 큽니다.\n"
        "히스토그램에서 관찰되는 중복 영역(종 간 겹침)은 종 간 경계가 완전하지 않음을 보여주며, 다변량(예: bill 길이/깊이와 결합) 분석이 분류 성능을 향상시킬 수 있습니다."
    ),
    '05_hist_bill_ratio.png': (
        "부리비(bill_ratio = bill_length_mm / bill_depth_mm)는 길이와 깊이의 상대적 비율로, 형태학적 차이를 더 민감하게 포착합니다.\n"
        "비율 분포는 종 간 구조적 차이를 강조할 수 있으며, 특정 종에서 평균 비율이 높은 경우 부리 형태가 길고 얇다는 것을 의미합니다.\n"
        "이 지표는 먹이 섭취 방식(예: 찌르기 vs 집어먹기)과 연관될 수 있으며, 단일 치수보다 더 해석적인 통찰을 제공합니다.\n"
        "비율의 분포를 섬과 성별로 분해하면 인구 내 형태학적 다양성과 지역적 적응 신호를 탐지하는 데 유용합니다."
    ),
    'box_body_mass_by_species.png': (
        "종별 체질량의 박스플롯은 중앙값, 사분위수 및 이상치를 동시에 보여주어 종 간 체질량 분포의 요약을 제공합니다.\n"
        "Gentoo는 중앙값과 사분위 범위가 높아 체질량이 전반적으로 크고 분산도 큰 편입니다. 반면 Adelie와 Chinstrap는 더 작은 중앙값과 상대적으로 좁은 사분위 범위를 보입니다.\n"
        "이상치는 개체 간 변동성 또는 표본 오류를 나타낼 수 있으므로 조사 대상에 포함하여 해석해야 합니다.\n"
        "박스플롯을 통해 종간 유의미한 체질량 차이가 관찰되면 보전 전략이나 생태적 역할 구분에서 중요한 단서를 제공합니다."
    ),
    'violin_bill_length_by_species.png': (
        "바이올린 플롯은 각 종 내 분포의 밀도를 시각화하여 데이터의 다봉성이나 분포 모양을 확인하는 데 유리합니다.\n"
        "예를 들어, 어떤 종은 부리 길이가 뚜렷한 중심을 갖는 반면, 다른 종은 넓은 분포를 보여 다양한 개체군 구성이 있음을 시사할 수 있습니다.\n"
        "바이올린 내부의 박스(중앙값과 IQR)를 함께 보면 중앙 경향과 분포 형태를 종합적으로 이해할 수 있습니다.\n"
        "종별 차이가 명확하면 형태학적 분류의 근거가 되고, 분포 내부의 다중 피크는 하위 집단(subpopulation)을 의심할 만한 신호입니다."
    ),
    'scatter_bill_length_depth_by_species.png': (
        "부리 길이와 깊이의 산점도는 두 변수가 종 간에 어떻게 함께 변하는지를 보여줍니다.\n"
        "종별로 색을 구분하면 각 종이 차별화된 클러스터를 형성하는지, 또는 영역이 겹치는지를 시각적으로 파악할 수 있습니다.\n"
        "일반적으로 종 간 클러스터가 있으면 이 두 변수로도 분류가 어느 정도 가능함을 의미하며, 겹침이 크면 추가 변수(예: flipper_length, body_mass)가 필요합니다.\n"
        "이 산점도는 형태학적 통합성(integration)과 상관구조를 이해하는 데 기본적이며, 이상치나 극단값을 확인해 후속 분석(제거/확인) 여부를 판단할 수 있습니다."
    ),
    'scatter_bill_vs_depth_size.png': (
        "사이즈(플리퍼 길이)를 점 크기로 표현한 산점도는 세 변수 간의 복합적 관계를 동시에 관찰할 수 있어 유용합니다.\n"
        "예를 들어, 동일한 부리 길이와 깊이 영역 내에서 플리퍼 길이가 큰 개체들이 특정 종에 편중되어 있다면, 이는 신체 크기와 부리 형태 사이의 통합성을 시사합니다.\n"
        "점 크기로 추가적인 정보를 주면 다변량 패턴을 직관적으로 탐색할 수 있으며, 이는 분포의 구조와 잠재적 상관관계를 초기 탐색하는 데 효과적입니다."
    ),
    'regplot_bill_length_vs_depth.png': (
        "회귀선을 포함한 산점도는 두 변수 간의 선형 관계를 평가하는 데 도움이 됩니다.\n"
        "회귀선의 기울기와 결정계수(R^2)는 부리 길이와 깊이 사이의 상관 강도를 정량적으로 나타내며, 약한 상관이면 두 변수가 독립적인 해석을 가능하게 합니다.\n"
        "잔차 패턴을 시각적으로 점검하면 비선형성이나 이분산성(heteroscedasticity)을 감지할 수 있고, 이는 모델링 선택(예: 비선형 회귀)을 안내합니다."
    ),
    'jointplot_bill_length_body_mass.png': (
        "조인트플롯은 산점도와 각 축의 히스토그램을 결합해 두 변수의 공분포와 각각의 단변량 분포를 동시에 제공합니다.\n"
        "부리 길이와 체질량의 관계를 통해 더 큰 부리를 가진 개체가 체질량이 큰지 여부를 시사할 수 있으며, 종별로 색을 넣으면 그룹 차이도 관찰할 수 있습니다.\n"
        "단변량 분포의 편향성이나 다봉성은 인구 구조(예: 나이/성별 구성)의 혼재를 나타낼 수 있으므로 추가 분할 분석이 필요합니다."
    ),
    'pairplot_numeric.png': (
        "페어플롯은 여러 수치형 변수의 쌍별 관계를 한 번에 보여주어 다변량 패턴과 변수 간 상관을 직관적으로 파악할 수 있게 합니다.\n"
        "대각선에는 각 변수의 분포가 표시되어 변수별 분포 특성을 확인할 수 있고, 비대각선에는 산점도가 있어 변수 쌍 간의 관계(선형/비선형, 클러스터 등)를 보여줍니다.\n"
        "전체적으로 플리퍼 길이와 체질량은 강한 양의 상관을 보이는 반면, 부리 길이/깊이와 체질량의 관계는 더 약한 패턴을 보일 수 있습니다.\n"
        "페어플롯은 변수 선택, 차원 축소, 분류 모델링에서 어떤 변수를 우선적으로 고려할지 결정하는 데 유용합니다."
    ),
    'heatmap_correlation.png': (
        "상관 히트맵은 수치형 변수들 간의 선형 상관계수를 매트릭스로 보여주며, 음수/양수 및 크기의 강도를 한눈에 파악할 수 있습니다.\n"
        "예를 들어, flipper_length와 body_mass는 비교적 높은 양의 상관을 보이며 이는 신체 크기와 관련된 통합적 특성임을 시사합니다.\n"
        "부리 길이와 깊이의 상관은 중간 수준일 수 있으며, 이는 두 변수가 완전히 독립적이지 않지만 서로를 대체할 수 없음을 의미합니다.\n"
        "상관 구조는 다중공선성 우려가 있는지, 변수 선택에서 중복을 제거해야 하는지를 판단할 때 중요합니다."
    ),
    'count_species.png': (
        "종별 개체 수 카운트플롯은 샘플링 분포와 종 간 상대적 빈도를 보여줍니다.\n"
        "데이터셋에서는 Adelie가 가장 많고 Chinstrap가 가장 적으며, 이는 분석 결과 해석 시 표본 크기 차이에 따른 통계적 힘(power)에 영향을 줍니다.\n"
        "표본 불균형은 일부 종에서 추정치의 신뢰구간이 넓어질 수 있으므로, 모델링 시 가중치 적용이나 재샘플링 기법을 고려해야 합니다."
    ),
    'grouped_bar_sex_species.png': (
        "성별과 종의 교차 카운트는 성비(성별 비율)가 종마다 어떻게 다른지를 보여줘 생태적·행동학적 시사점을 제공합니다.\n"
        "만약 특정 종에서 한 성별이 상대적으로 우세하다면 번식 전략 또는 표본 수집 방법의 편향을 의심해볼 수 있습니다.\n"
        "이 바 차트와 함께 교차표(crosstab)를 검토하면 절대빈도와 비율을 모두 확인할 수 있어 더 균형 잡힌 해석이 가능합니다."
    ),
    'stacked_bar_island_species.png': (
        "섬별 종 구성의 스택형 막대그래프는 지역별 종 조성(composition)을 비율로 비교하는 데 적합합니다.\n"
        "섬 간에 특정 종이 우세한 경우, 지역적 서식지 특성 또는 먹이자원 분포가 종 분포에 영향을 미친다는 가설을 세울 수 있습니다.\n"
        "이 시각화는 보전 우선 순위, 지역별 서식지 관리 계획 및 추가 현장 조사 지점을 결정하는 데 도움을 줍니다."
    ),
    'crosstab_species_island.md': (
        "교차표(species x island)는 각 섬에서 어떤 종이 얼마나 표본화되었는지를 표 형식으로 정리해 줍니다.\n"
        "이 표로부터 특정 섬에서 한 종이 우세한지, 또는 여러 종이 고르게 분포하는지를 파악할 수 있습니다.\n"
        "교차표를 비율로 변환하면 표본 수의 차이를 보정한 비교가 가능하고, Chi-square 등 통계 검정을 통해 유의미한 분포 차이가 있는지 확인할 수 있습니다."
    ),
    'pivot_island_species_bodymass.md': (
        "피봇테이블(mean body_mass_g by island x species)은 각 섬에서 종별 평균 체질량을 요약하여 지역별 생리적/영양 상태의 차이를 보여줍니다.\n"
        "예를 들어 같은 종이라도 섬 간 평균 체질량 차이가 있다면 먹이자원, 기후, 번식 상태 등 환경적 요인이 영향을 미쳤을 가능성이 있습니다.\n"
        "평균과 함께 표준편차/표준오차를 추가하면 차이의 통계적 유의성을 판단하는 데 도움이 되며, 시각적으로는 heatmap이나 바 차트로 쉽게 비교할 수 있습니다."
    )
}

# Provide a default short insight if a file is not covered
default_insight = "해당 이미지에 대한 요약 인사이트가 준비되어 있지 않습니다. 필요하면 수동으로 추가해 주세요."
print('Insight registry created for', len(insights), 'items')

In [ ]:
# 6) Quick verification tests: >=10 images and md content
imgs = sorted(glob.glob('figures/*.png'))
assert len(imgs) >= 10, f'Expected >=10 images, found {len(imgs)}'
print('Image count OK:', len(imgs))

with open('penguins_analysis.md', 'r', encoding='utf-8') as f:
    content = f.read()
assert '교차표' in content or 'Crosstab' in content, 'Crosstab not found in markdown'
assert '피봇테이블' in content or 'pivot' in content, 'Pivot not found in markdown'
print('Markdown content verification OK')

# 7) (Optional) show a sample of the markdown
print('\n--- Start of penguins_analysis.md snippet ---\n')
print('\n'.join(content.splitlines()[:40]))
print('\n--- End of snippet ---')